In [1]:
import socket
import sys
import os
import tkinter as tk
from tkinter import *

In [16]:
def connectFunc():
    content = f"You Entered <CONNECT>\nConnecting to {serverAddress[0]} port {serverAddress[1]}...\n"
    try:
        sock.connect(serverAddress)
        content += "Connection successful!"
    except Exception as e:
        content += f"Connection failed. Error: {e}"
    finally:
        text_widget.delete("1.0", tk.END) 
        text_widget.insert(tk.END, content)
        text_widget.tag_add("my_tag", "1.0", "end")
        
        print(content)
    
    
def helpFunc():
    content = """
    HELP                                   : Showing the list of commands
    CONNECT                          : Connect to FTP server
    UPLOAD <file_path>         : Upload file from the specified path
    LIST                                   : List files available on the server
    DOWNLOAD <file_path> : Download file from the specified path
    DELETE <file_path>          : Delete file from the specified path
    PWD                                   : Showing your current location in the server
    QUIT                                 : Ending the connection to the server
    """
    text_widget.insert(tk.END, content)
    text_widget.tag_add("my_tag", "1.0", "end")
    
    print("Enter one of the following commands:\nHELP                 : Showing the list of commands\nCONNECT              : Connect to FTP server\nUPLOAD <file_path>   : Upload file from the specified path\nLIST                 : List files\nDOWNLOAD <file_path> : Download file from the specified path\nDELETE <file_path>   : Delete file from the specified path\nPWD                  : Showing our current locaiton in the server\nQUIT                 : Ending the connection to server")
    
def uploadFunc(file_path):
    file_path = file_path.replace('"', '')
    if not file_path:
        content = 'Please enter message in the appropriate format.\n'
        text_widget.delete("1.0", tk.END)
        text_widget.insert(tk.END, content)
        text_widget.tag_add("my_tag", "1.0", "end")
        print(content)
    else:
        if os.path.isfile(file_path):
            file_name = os.path.basename(file_path)
            sock.sendall(f'UPLOAD {file_name}'.encode())
            sock2 = socket.socket()
            sock2.connect(('localhost', 2222))
            content = "Connection established with port 2222.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
            with open(file_path, 'rb') as file:
                file_data = file.read(1024)
                while file_data:
                    sock2.sendall(file_data)
                    file_data = file.read(1024)
                content = '---DONE---\n'
                text_widget.insert(tk.END, content)
                text_widget.tag_add("my_tag", "1.0", "end")
                print(content)
            sock2.close()

def listFilesFunc():
    sock.sendall('LIST'.encode())
    listOfFiles = sock.recv(1024)
    content = 'List of files:\n' + listOfFiles.decode()
    text_widget.insert(tk.END, content)
    text_widget.tag_add("my_tag", "1.0", "end")
    print(content)
    
def downloadFilesFunc(file_path):
    client_directory = r"C:\Users\parin\Desktop\NetworkFinalProject\clientFolder"
    file_path = file_path.replace('"', '')  
    file_name = os.path.basename(file_path)
    if not file_path:
        content = 'Please enter message in the appropriate format.'
        text_widget.insert(tk.END, content)
        text_widget.tag_add("my_tag", "1.0", "end")
        print(content)
    else:
        if os.path.isfile(file_path):
            sock.sendall(f'DOWNLOAD {file_path}'.encode())
            sock2 = socket.socket()
            sock2.connect(('localhost', 2222))
            content = "New connection with port 2222 stablished.\nSending data now...\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
            client_file_path = os.path.join(client_directory, file_name)
            success = False
            with open(client_file_path, 'wb') as file:
                while True:
                    file_data = sock2.recv(1024)
                    if file_data == b"File not found":
                        content = "File not found on the server\n"
                        text_widget.insert(tk.END, content)
                        text_widget.tag_add("my_tag", "1.0", "end")
                        print(content)
                        break
                    file.write(file_data)
                    if len(file_data) < 1024:
                        success = True
                        break
            content = f"File '{file_name}' downloaded to '{client_directory}' successfully.\n--- DONE ---\n" if success else f"--- File Not Found! ---"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
            sock2.close()
        else:
            content = "A file with this path does not exist!\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
                
    
def deleteFilesFunc(path):
    content = 'The path to delete was ' + path
    text_widget.insert(tk.END, content)
    text_widget.tag_add("my_tag", "1.0", "end")
    print(content)
    path = 'DELETE ' + path
    sock.sendall(path.encode())
    
def PWDFunc():
    sock.sendall('PWD'.encode())
    directory = sock.recv(1024)
    content = 'The current working directory is:\t' + directory.decode()
    text_widget.insert(tk.END, content)
    text_widget.tag_add("my_tag", "1.0", "end")
    print(content)

In [ ]:
def on_enter_key(event):
    OKbutton.invoke()

def OKclick():
    global connected
    prompt = e.get()
    e.delete(0, tk.END)
    text_widget.delete("1.0", tk.END)
    
    if prompt[:4].upper() == "HELP": helpFunc()
        
    elif prompt[:7].upper() == "CONNECT":
        if connected:
            content = "You are already connected to port 2121.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
        else:
            connectFunc()
            connected = True
        
    elif prompt[:8].upper() == 'SHUTDOWN':
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
        else:
            sock.sendall('SHUTDOWN'.encode())
            root.destroy()
            
    elif prompt[:6].upper() == "UPLOAD":
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
        else:
            uploadFunc(prompt[8:len(prompt)-1])
            
    elif prompt[:4].upper() == "LIST":
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
        else:
            listFilesFunc()

    elif prompt[:8].upper() == "DOWNLOAD":
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
        else:
            downloadFilesFunc(prompt[10:len(prompt)-1])
        
    elif prompt[:6].upper() == "DELETE":
        path = prompt[8:len(prompt)-1]
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
        elif not path:
            content = 'Please enter path in the appropriate format.\n'
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
            print(content)
        else:
            deleteFilesFunc(path)
            
    elif prompt[:3].upper() == "PWD":
        if not connected:
            content = "You are not connected to FTP server! Enter 'CONNECT' message.\n"
            text_widget.insert(tk.END, content)
            text_widget.tag_add("my_tag", "1.0", "end")
        else:
            PWDFunc()
        
    elif prompt[:4].upper() == "QUIT": root.destroy()

    else:
        content = "You entered <" + prompt + '>' + '\nInvalid message. Please try again.\n'
        text_widget.insert(tk.END, content)
        text_widget.tag_add("my_tag", "1.0", "end")
        print (content)

root = tk.Tk()
root.title("FTP Client")
root.geometry("700x400")

welcometext = Label(root, text='------ YOUR FTP CLIENT ------\n', font=('Times New Roman', 12))
welcometext.pack()
text_widget = Text(root, wrap="word", height=13, width=70)
text_widget.pack()
scrollbar = Scrollbar(root, command=text_widget.yview)
scrollbar.pack(side="right", fill="y")
text_widget.config(yscrollcommand=scrollbar.set)
# Setting the font size and family
font_size = 11
text_widget.tag_configure("my_tag", font=("Comic Sans MS", font_size))

# Initial content
helpFunc()
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverAddress = ('localhost', 2121)
connected = False

enterText = Label(root, text="Enter your message: ", font=('Times New Roman', 12))

enterText.pack()
e = Entry(root, width=50, bd=3, bg='#ADD8E6', fg='#02075D', borderwidth=3, font=('cursive', 12)) #Courier
e.pack()

prompt_label = Label(root, text='')
prompt_label.pack()

OKbutton = Button(root, text='Send', padx=4, pady=2, command=OKclick, fg="white", bg="green")
OKbutton.pack()
root.bind('<Return>', on_enter_key)

root.mainloop()
sock.close()
print("***EXITED***")

Enter one of the following commands:
HELP                 : Showing the list of commands
CONNECT              : Connect to FTP server
UPLOAD <file_path>   : Upload file from the specified path
LIST                 : List files
DOWNLOAD <file_path> : Download file from the specified path
DELETE <file_path>   : Delete file from the specified path
PWD                  : Showing our current locaiton in the server
QUIT                 : Ending the connection to server


# Graphics Tutorial

In [9]:
#Label
root = Tk()
root.title("FTP Client")
myLabel = Label(root, text = "Hello world!").grid(row=0, column=0)
myLabel2 = Label(root, text = "I'm Parinaz Akef.").grid(row=1, column=5)
root.mainloop()

In [54]:
#Button
root = Tk()
def myClick():
    myLabel = Label(root, text="Look! I clicked a Button!!").pack()
myButton = Button(root, text = 'OK', padx=5, pady=2, command=myClick, fg="white", bg="green")
myButton.pack()
root.mainloop()

In [37]:
#input

root = Tk()
root.geometry("500x200")
e = Entry(root, width=50, bg='black', fg='yellow', borderwidth=3)
e.pack()
e.insert(0, "Enter your name: ")
def myClick():
    myLabel = Label(root, text='Hello ' + e.get()).pack()
myButton = Button(root, text="Enter", command=myClick).pack()
root.mainloop()

In [78]:
import tkinter as tk

root = tk.Tk()
root.geometry("500x200")

text_widget = tk.Text(root, wrap="none")
text_widget.pack(side="left", fill="both", expand=True)

scrollbar = tk.Scrollbar(root, command=text_widget.yview)
scrollbar.pack(side="right", fill="y")
text_widget.config(yscrollcommand=scrollbar.set)

for i in range(20):
    text_widget.insert("end", f"Line {i}\n")

root.mainloop()

In [138]:
import tkinter as tk
from tkinter import Label, Entry, Button, Text, Scrollbar

def helpFunc():
    content = """
    HELP: Showing the list of commands
    CONNECT: Connect to FTP server
    UPLOAD <file_path>: Upload file from the specified path
    LIST: List files
    DOWNLOAD <file_path>: Download file from the specified path
    DELETE <file_path>: Delete file from the specified path
    PWD: Showing our current location in the server
    QUIT: Ending the connection to server
    """
    text_widget.delete("1.0", tk.END) 
    text_widget.insert(tk.END, content)

def OKclick():
    prompt = e.get()
    if prompt[:4].upper() == "HELP": helpFunc()
    else:
        content = "You entered " + prompt
        text_widget.delete("1.0", tk.END) 
        text_widget.insert(tk.END, content)

root = tk.Tk()
root.title("FTP Client")
root.geometry("800x500")

welcometext = Label(root, text='------WELCOME TO FTP CLIENT------\n')
welcometext.pack()
text_widget = Text(root, wrap="word", height=10, width=70)
text_widget.pack()
scrollbar = Scrollbar(root, command=text_widget.yview)
scrollbar.pack(side="right", fill="y")
text_widget.config(yscrollcommand=scrollbar.set)

# Initial content
helpFunc()
connected = False
enterText = Label(root, text="Enter your message: ")
enterText.pack()
e = Entry(root, width=50, bg='blue', fg='white', borderwidth=3)
e.pack()
prompt_label = Label(root, text='')
prompt_label.pack()
OKbutton = Button(root, text='Send', padx=4, pady=2, command=OKclick, fg="white", bg="green")
OKbutton.pack()

root.mainloop()